# Camshift Tracking with OpenCV Implementation:

In [2]:
import numpy as np
import cv2

In [3]:
cap = cv2.VideoCapture(0)


ret,frame = cap.read()

# we will start by setting our initial tracking window...

# Previously we were using corner detection to do this just detecting 10 best corners on the first frame and tracking
# those as we went along.

# lets actually now perform face tracking.
# To do this, we are gonna perform object detection on the very first frame, to grab the face location
# then we actually have detect the face we are gonna treat that as a bunch of pixels that we are meaning to track
# and then we apply meanshift tracking on that face.

# so we are detecting the face one time at the very beginning and telling the meanshift tracking algorithm to 
# track that particular set of pixels


face_cascade = cv2.CascadeClassifier(r"C:\Users\JERRY\Downloads\Computer-Vision-with-Python\Computer-Vision-with-Python\DATA\haarcascades\haarcascade_frontalface_default.xml")

# essentially this returns back those corners
face_rects = face_cascade.detectMultiScale(frame)
# This essentially returns a list of all the numpy arrays where it is detecting a face.

# our intent is to detect only one face , so lets grab/use the first face it detects.


# Keep in mind, we are only going to be able to track one single face using this exact code.

(face_x,face_y,w,h) = tuple(face_rects[0])   # we are conveting it in to tuple because it is the required form to proceed furhter

track_window = (face_x,face_y,w,h) # this is how we can refer to these points as tracking window

# the next step is to set up a ROI for tracking...

roi = frame[face_y:face_y+h,face_x:face_x+w]

# now we are going to use HSV color mapping

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

# so now we are gonna find a histogram to back project the target on each frame in order to calculate that ]
# meanshift.

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# now we normalize histogram array values

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# now we are settting up the termination criteria...
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret,frame = cap.read()
    
    if ret == True:  # if this is true, we are gonna grab the frame and convert it to HSV
       
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        # Now we need to calculate the back projection based of the ROI hist that we created earlier.
        
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        
        
        ###############################################################
        ret,track_window = cv2.CamShift(dst,track_window,term_crit)
        
        # and now we will draw on the image absed off the points
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)  # turning them all into integers... coz by default it returns floating point.
        
        # we will draw  some polynomial lines
        img2 = cv2.polylines(frame,[pts],True,(0,0,255),5)
        
    
        
        
        ###############################################################
        cv2.imshow('img',img2)
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            
            break
            
    else:
        break
        

cap.release()
cv2.destroyAllWindows()